## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN'.
df = df.drop(columns=["EIN"])

In [4]:
# Determine the number of unique values in each column.
# Columns 10+ unique values: NAME, APPLICATION_TYPE, CLASSIFICATION
df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
# Look at NAME value counts for binning
name = df["NAME"].value_counts()
name[:5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
Name: NAME, dtype: int64

In [6]:
# Create a replaced cutoff value for "NAME" as `name_to_replace`
cutoff = 5
name_to_replace = name[name < cutoff].index.tolist()

# Replace in dataframe
for n in name_to_replace:
    df['NAME'] = df['NAME'].replace(n,"Other")

# Check to make sure binning was successful
df['NAME'].value_counts()[:5]

Other                                 19803
PARENT BOOSTER USA INC                 1260
TOPS CLUB INC                           765
UNITED STATES BOWLING CONGRESS INC      700
WASHINGTON STATE UNIVERSITY             492
Name: NAME, dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
app_type = df["APPLICATION_TYPE"].value_counts()
app_type[:9]

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 276
application_types_to_replace = app_type[app_type < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification = df["CLASSIFICATION"].value_counts()
classification[:5]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_gt1 = classification[classification >1]
classification_gt1[:5]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1883
classifications_to_replace = classification[classification < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df)

In [13]:
# Split our preprocessed data into our features and target arrays
# 21.2.7
y = df["IS_SUCCESSFUL"].values
X = df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#21.2.6
#21.12.7
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  130
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 50 

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 130)               58110     
                                                                 
 dense_1 (Dense)             (None, 80)                10480     
                                                                 
 dense_2 (Dense)             (None, 50)                4050      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 72,691
Trainable params: 72,691
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.4646 - accuracy: 0.7808
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4302 - accuracy: 0.7958
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4255 - accuracy: 0.7970
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4224 - accuracy: 0.7986
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4212 - accuracy: 0.7986
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4205 - accuracy: 0.8014
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4192 - accuracy: 0.8012
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4187 - accuracy: 0.8001
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4178 - accuracy: 0.8021
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4168 - accuracy: 0.8015

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5034 - accuracy: 0.7913 - 513ms/epoch - 2ms/step
Loss: 0.5033594369888306, Accuracy: 0.7912536263465881


In [19]:
# Export our model to HDF5 file
# nn.save("Model/AlphabetSoupCharity_Optimization.h5")